In [ ]:
import os
os.chdir('../input/pretrainmodels')
!python ./setup.py install

In [ ]:
from glob import glob

import torch
import pandas as pd
from PIL import Image, ImageFile
from torchvision import transforms


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import pretrainedmodels
os.chdir('../../working')

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'


def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)


def get_se_resnet50_gem(pretrain):
    if pretrain == 'imagenet':
        model = pretrainedmodels.__dict__['se_resnet50'](num_classes=1000, pretrained='imagenet')
    else:
        model = pretrainedmodels.__dict__['se_resnet50'](num_classes=1000, pretrained=None)
    model.avg_pool = GeM()
    model.last_linear = nn.Linear(2048, 1)
    return model

MODEL_PATH = '../input/224best/224best.pth'
MODEL_PATH_list = ['../input/224best/224best.pth',
                   '../input/128best/128best.pth',
                   '../input/seresnet384/model_epoch33.pth']
resize_list = [224,128,384]
TEST_IMAGE_PATH = '../input/aptos2019-blindness-detection/test_images'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


test_images = glob(os.path.join(TEST_IMAGE_PATH, '*.png'))
predictions = []
for i, im_path in enumerate(test_images):
    predictions.append([os.path.splitext(im_path.split('/')[-1])[0], 0])

# Load model
for idx in range(len(MODEL_PATH_list)):
    model = get_se_resnet50_gem(pretrain=None)
    model.to(device)
    model.load_state_dict(torch.load(MODEL_PATH_list[idx]))
    model.eval()

    # Inference
    for i, im_path in enumerate(test_images):
        print(i, '/', len(test_images))
        image = Image.open(im_path)
        image = image.resize((resize_list[idx], resize_list[idx]), resample=Image.BILINEAR)
        image = transforms.ToTensor()(image).to(device)
        output = model(image.unsqueeze(0))

        # Make submission
        predictions[i][1] = (predictions[i][1]*idx + output.item())/(idx+1)

submission = pd.DataFrame(predictions)
submission.columns = ['id_code','diagnosis']
submission.to_csv('submission_raw_value.csv', index=False)
submission.head()

submission.loc[submission.diagnosis < 0.7, 'diagnosis'] = 0
submission.loc[(0.7 <= submission.diagnosis) & (submission.diagnosis < 1.5), 'diagnosis'] = 1
submission.loc[(1.5 <= submission.diagnosis) & (submission.diagnosis < 2.5), 'diagnosis'] = 2
submission.loc[(2.5 <= submission.diagnosis) & (submission.diagnosis < 3.5), 'diagnosis'] = 3
submission.loc[3.5 <= submission.diagnosis, 'diagnosis'] = 4
submission['diagnosis'] = submission['diagnosis'].astype(int)
submission.to_csv('submission.csv', index=False)
submission.head()